In [1]:
import os

import openmm as mm
from openmm.app import *
from openmm import unit
from openmm import app
from pdbfixer import PDBFixer
import mdtraj as md
import nglview as nv
import requests
import numpy as np
from math import sqrt
import itertools

# Barnase-Barstar Simulation

## System Analysis

In [2]:
# Function to download PDB files

def fetch_pdb(pdb_id, download_path="./"):

        url = 'http://files.rcsb.org/download/{}.pdb'.format(pdb_id)
        try:
            res = requests.get(url, allow_redirects=True)
        except:
            print("Could not fetch pdb from {}".format(url))
            return 
        
        file_path = os.path.join(download_path, pdb_id + ".pdb")
        with open(file_path, "wb") as f:
            f.write(res.content)

In [3]:
# Just write the PDB id in order to download the pdb file

fetch_pdb("1brs")

In [4]:
# Load pdb with MDTraj

brs = md.load('1brs.pdb')

In [5]:
# Remove water molecules

brs = brs.remove_solvent()

In [6]:
brs.topology

<mdtraj.Topology with 6 chains, 588 residues, 4638 atoms, 4740 bonds at 0x7f0ccc920df0>

In [7]:
# Barnase's chains

atoms_in_chain_A = brs.topology.select("chainid == 0")
atoms_in_chain_B = brs.topology.select("chainid == 1")
atoms_in_chain_C = brs.topology.select("chainid == 2")

In [8]:
# Barstar's chains

atoms_in_chain_D = brs.topology.select("chainid == 3")
atoms_in_chain_E = brs.topology.select("chainid == 4")
atoms_in_chain_F = brs.topology.select("chainid == 5")

In [9]:
# Barnase's chain's atoms

barnase_A = brs.atom_slice(atoms_in_chain_A)
barnase_B = brs.atom_slice(atoms_in_chain_B)
barnase_C = brs.atom_slice(atoms_in_chain_C)

In [10]:
# Barstar's chain's atoms

barstar_D = brs.atom_slice(atoms_in_chain_D)
barstar_E = brs.atom_slice(atoms_in_chain_E)
barstar_F = brs.atom_slice(atoms_in_chain_F)

In [11]:
print(f'Barnase has {barnase_A.n_atoms} atoms and {barnase_A.n_residues} residues in A')
print(f'Barnase has {barnase_B.n_atoms} atoms and {barnase_B.n_residues} residues in B')
print(f'Barnase has {barnase_C.n_atoms} atoms and {barnase_C.n_residues} residues in C')

Barnase has 864 atoms and 108 residues in A
Barnase has 878 atoms and 110 residues in B
Barnase has 839 atoms and 108 residues in C


In [12]:
print(f'Barstar has {barstar_D.n_atoms} atoms and {barstar_D.n_residues} residues in D')
print(f'Barstar has {barstar_E.n_atoms} atoms and {barstar_E.n_residues} residues in E')
print(f'Barstar has {barstar_F.n_atoms} atoms and {barstar_F.n_residues} residues in F')

Barstar has 693 atoms and 87 residues in D
Barstar has 665 atoms and 86 residues in E
Barstar has 699 atoms and 89 residues in F


<div class="alert alert-info">
<strong>NOTE:</strong> We conclude chain B and F will be optimal to work.
</div>


In [13]:
view = nv.show_mdtraj(brs)
view

NGLWidget()

<div class="alert alert-info">
<strong>NOTE:</strong> Superpose F with E.
</div>

## System Preparation

In [14]:
ca_in_chain_E = brs.topology.select("chainid == 4 and name CA")
ca_in_chain_E

array([3275, 3284, 3289, 3296, 3304, 3312, 3316, 3325, 3334, 3342, 3349,
       3355, 3363, 3369, 3377, 3385, 3395, 3400, 3407, 3415, 3424, 3433,
       3442, 3450, 3455, 3463, 3470, 3479, 3491, 3503, 3507, 3516, 3524,
       3532, 3540, 3545, 3553, 3567, 3575, 3580, 3588, 3595, 3599, 3613,
       3620, 3629, 3641, 3648, 3656, 3663, 3671, 3680, 3694, 3701, 3710,
       3721, 3730, 3735, 3741, 3746, 3751, 3756, 3763, 3767, 3772, 3781,
       3787, 3794, 3802, 3811, 3818, 3829, 3840, 3849, 3854, 3863, 3868,
       3877, 3881, 3886, 3894, 3902, 3909, 3917, 3925, 3933])

In [15]:
ca_in_chain_F = brs.topology.select("chainid == 5 and name CA")
ca_in_chain_F

array([3940, 3949, 3958, 3963, 3970, 3978, 3986, 3990, 3999, 4008, 4016,
       4027, 4033, 4041, 4047, 4055, 4063, 4073, 4082, 4089, 4097, 4106,
       4112, 4121, 4129, 4134, 4142, 4149, 4154, 4166, 4178, 4182, 4191,
       4199, 4207, 4215, 4220, 4228, 4242, 4250, 4255, 4263, 4270, 4274,
       4288, 4295, 4300, 4312, 4319, 4327, 4334, 4342, 4351, 4365, 4376,
       4385, 4396, 4405, 4414, 4420, 4429, 4438, 4446, 4453, 4458, 4463,
       4467, 4472, 4481, 4487, 4494, 4502, 4511, 4518, 4529, 4540, 4549,
       4554, 4563, 4568, 4577, 4581, 4586, 4594, 4602, 4609, 4617, 4625,
       4633])

In [16]:
chain_E = brs.atom_slice(ca_in_chain_E)
chain_F = brs.atom_slice(ca_in_chain_F)

In [17]:
ca_name_in_E = []
for atom in chain_E.topology.atoms_by_name('CA'):
    ca_name_in_E.append(str(atom))

In [18]:
ca_name_in_F = []
for atom in chain_F.topology.atoms_by_name('CA'):
    ca_name_in_F.append(str(atom))

In [19]:
index = []

for x in ca_name_in_F:    #Lista grande
    for y in ca_name_in_E:    #Lista pequeña
        if x in y:
            index.append(True)
            break
    else:
        index.append(False)
            
len(index)

89

In [20]:
print(index)

[False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [21]:
ca_in_chain_G = []

for uu in range(len(index)):
    if index[uu] == True:
        ca_in_chain_G.append(ca_in_chain_F[uu])
        
ca_in_chain_G = np.array(ca_in_chain_G)

In [22]:
ca_in_chain_G

array([3949, 3958, 3963, 3970, 3978, 3986, 3990, 3999, 4008, 4016, 4027,
       4033, 4041, 4047, 4055, 4063, 4073, 4082, 4089, 4097, 4106, 4112,
       4121, 4129, 4134, 4142, 4149, 4154, 4166, 4178, 4182, 4191, 4199,
       4207, 4215, 4220, 4228, 4242, 4250, 4255, 4263, 4270, 4274, 4288,
       4295, 4300, 4312, 4319, 4327, 4334, 4342, 4351, 4365, 4376, 4385,
       4396, 4405, 4414, 4420, 4429, 4438, 4446, 4463, 4467, 4472, 4481,
       4487, 4494, 4502, 4511, 4518, 4529, 4540, 4549, 4554, 4563, 4568,
       4577, 4581, 4586, 4594, 4602, 4609, 4617, 4625, 4633])

In [23]:
traj_E = brs.atom_slice(ca_in_chain_E)
traj_E

<mdtraj.Trajectory with 1 frames, 86 atoms, 86 residues, and unitcells at 0x7f0cc1e53eb0>

In [24]:
traj_F = brs.atom_slice(ca_in_chain_G)
traj_F

<mdtraj.Trajectory with 1 frames, 86 atoms, 86 residues, and unitcells at 0x7f0cc1e5e310>

In [25]:
brstr_to_B = md.Trajectory.superpose(traj_F, traj_E, frame=0)

In [26]:
pp = nv.show_mdtraj(brstr_to_B)
pp

NGLWidget()

In [27]:
qq = nv.show_mdtraj(barnase_B)
qq

NGLWidget()

In [28]:
brns_brstr = barnase_B.stack(brstr_to_B)

In [29]:
uu = nv.show_mdtraj(brns_brstr)
uu

NGLWidget()

In [30]:
brns_brstr.save_pdb('brns_brstr.pdb')

In [31]:
help(md.Trajectory.save_pdb)

Help on function save_pdb in module mdtraj.core.trajectory:

save_pdb(self, filename, force_overwrite=True, bfactors=None)
    Save trajectory to RCSB PDB format
    
    Parameters
    ----------
    filename : path-like
        filesystem path in which to save the trajectory
    force_overwrite : bool, default=True
        Overwrite anything that exists at filename, if its already there
    bfactors : array_like, default=None, shape=(n_frames, n_atoms) or (n_atoms,)
        Save bfactors with pdb file. If the array is two dimensional it should
        contain a bfactor for each atom in each frame of the trajectory.
        Otherwise, the same bfactor will be saved in each frame.



In [32]:
help(PDBFixer)

Help on class PDBFixer in module pdbfixer.pdbfixer:

class PDBFixer(builtins.object)
 |  PDBFixer(filename=None, pdbfile=None, pdbxfile=None, url=None, pdbid=None)
 |  
 |  PDBFixer implements many tools for fixing problems in PDB and PDBx/mmCIF files.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, filename=None, pdbfile=None, pdbxfile=None, url=None, pdbid=None)
 |      Create a new PDBFixer instance to fix problems in a PDB or PDBx/mmCIF file.
 |      
 |      Parameters
 |      ----------
 |      filename : str, optional, default=None
 |          The name of the file to read.  The format is determined automatically based on the filename extension, or if
 |          that is ambiguous, by looking at the file content.
 |      pdbfile : file, optional, default=None
 |          A file-like object from which the PDB file is to be read.
 |          The file is not closed after reading.
 |      pdbxfile : file, optional, default=None
 |          A file-like object from which the PDB

In [33]:
fixer = PDBFixer(filename='brns_brstr.pdb')

In [34]:
fixer.topology

<Topology; 2 chains, 196 residues, 964 atoms, 899 bonds>

In [35]:
fixer.findMissingResidues()
missing_residues = fixer.missingResidues
print(f"{len(missing_residues)} missing residues")

fixer.findNonstandardResidues()
nonstandard_residues = fixer.nonstandardResidues
print(f"{len(nonstandard_residues)} non standard residues")

fixer.findMissingAtoms()
missing_atoms = fixer.missingAtoms
missing_terminals = fixer.missingTerminals
print(f"{len(missing_atoms)} missing atoms")
print(f"{len(missing_terminals)} missing terminals")

if len(nonstandard_residues)>0:
    fixer.replaceNonstandardResidues()

if len(missing_atoms)>0:
    fixer.addMissingAtoms()

0 missing residues
0 non standard residues
86 missing atoms
1 missing terminals


In [36]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [37]:
modeller = app.Modeller(fixer.topology, fixer.positions)
pH = 7.2
residues_protonated = modeller.addHydrogens(forcefield=forcefield, pH=pH)

In [38]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.NoCutoff)

charge = 0.0 * unit.elementary_charge
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, mm.NonbondedForce):
        for index in range(system.getNumParticles()):
            charge+=force.getParticleParameters(int(index))[0]

charge = np.round(charge._value)*charge.unit

In [39]:
charge

Quantity(value=-4.0, unit=elementary charge)

In [40]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, open('brs_full.pdb', 'w'))

In [41]:
uu = md.load("brs_full.pdb")
view = nv.show_mdtraj(uu)
view

NGLWidget()

In [42]:
# Solvate system

pdb = app.PDBFile('brs_full.pdb')
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')
modeller = app.Modeller(pdb.topology, pdb.positions)

In [43]:
geompadding = 1.4 * unit.nanometers
maxSize = max(max((pos[i] for pos in pdb.positions))-min((pos[i] for pos in pdb.positions)) for i in range(3))
vectors = mm.Vec3(1,0,0), mm.Vec3(1/3,2*sqrt(2)/3,0), mm.Vec3(-1/3,sqrt(2)/3,sqrt(6)/3)
boxVectors = [(maxSize+geompadding)*v for v in vectors]

In [44]:
modeller.addSolvent(forcefield, model='tip3p', boxVectors=boxVectors)

In [45]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, open('brns-solv.pdb', 'w'))

In [46]:
uu = md.load_pdb('brns-solv.pdb')
view = nv.show_mdtraj(uu)
view.add_licorice(selection='water')
view

NGLWidget()

## System Minimization